In [18]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir(r'C:\Users\Hp\OneDrive\Desktop\human activity\data set\train')

label_types = os.listdir(r'C:\Users\Hp\OneDrive\Desktop\human activity\data set\train')
print (label_types)  

['dancing', 'exercise', 'yoga']


In [19]:
rooms = []

for item in dataset_path:
 all_rooms = os.listdir(r'C:\Users\Hp\OneDrive\Desktop\human activity\data set\train' + '/' +item)

 for room in all_rooms:
    rooms.append((item, str(r'C:\Users\Hp\OneDrive\Desktop\human activity\data set\train' + '/' +item) + '/' + room))
          
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

       tag                                         video_name
0  dancing  C:\Users\Hp\OneDrive\Desktop\human activity\da...
1  dancing  C:\Users\Hp\OneDrive\Desktop\human activity\da...
2  dancing  C:\Users\Hp\OneDrive\Desktop\human activity\da...
3  dancing  C:\Users\Hp\OneDrive\Desktop\human activity\da...
4  dancing  C:\Users\Hp\OneDrive\Desktop\human activity\da...
     tag                                         video_name
13  yoga  C:\Users\Hp\OneDrive\Desktop\human activity\da...
14  yoga  C:\Users\Hp\OneDrive\Desktop\human activity\da...
15  yoga  C:\Users\Hp\OneDrive\Desktop\human activity\da...
16  yoga  C:\Users\Hp\OneDrive\Desktop\human activity\da...
17  yoga  C:\Users\Hp\OneDrive\Desktop\human activity\da...


In [20]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

In [21]:
dataset_path = os.listdir(r'C:\Users\Hp\OneDrive\Desktop\human activity\data set\test')
print(dataset_path)

room_types = os.listdir(r'C:\Users\Hp\OneDrive\Desktop\human activity\data set\test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 all_rooms = os.listdir(r'C:\Users\Hp\OneDrive\Desktop\human activity\data set\test' + '/' +item)

 for room in all_rooms:
    rooms.append((item, str(r'C:\Users\Hp\OneDrive\Desktop\human activity\data set\test' + '/' +item) + '/' + room))
    
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['dancing', 'exercise', 'yoga']
Types of activities found:  3
        tag                                         video_name
0   dancing  C:\Users\Hp\OneDrive\Desktop\human activity\da...
1   dancing  C:\Users\Hp\OneDrive\Desktop\human activity\da...
2   dancing  C:\Users\Hp\OneDrive\Desktop\human activity\da...
3  exercise  C:\Users\Hp\OneDrive\Desktop\human activity\da...
4  exercise  C:\Users\Hp\OneDrive\Desktop\human activity\da...
        tag                                         video_name
4  exercise  C:\Users\Hp\OneDrive\Desktop\human activity\da...
5  exercise  C:\Users\Hp\OneDrive\Desktop\human activity\da...
6      yoga  C:\Users\Hp\OneDrive\Desktop\human activity\da...
7      yoga  C:\Users\Hp\OneDrive\Desktop\human activity\da...
8      yoga  C:\Users\Hp\OneDrive\Desktop\human activity\da...


In [22]:

from tensorflow import keras
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [23]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

In [25]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 18
Total videos for testing: 9


,Unnamed: 0,video_name,tag
10,10,C:\Users\Hp\OneDrive\Desktop\human activity\da...,exercise
12,12,C:\Users\Hp\OneDrive\Desktop\human activity\da...,yoga
11,11,C:\Users\Hp\OneDrive\Desktop\human activity\da...,exercise
15,15,C:\Users\Hp\OneDrive\Desktop\human activity\da...,yoga
3,3,C:\Users\Hp\OneDrive\Desktop\human activity\da...,dancing
9,9,C:\Users\Hp\OneDrive\Desktop\human activity\da...,exercise
0,0,C:\Users\Hp\OneDrive\Desktop\human activity\da...,dancing
16,16,C:\Users\Hp\OneDrive\Desktop\human activity\da...,yoga
5,5,C:\Users\Hp\OneDrive\Desktop\human activity\da...,dancing
17,17,C:\Users\Hp\OneDrive\Desktop\human activity\da...,yoga


In [26]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [27]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [28]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['dancing', 'exercise', 'yoga']


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2]], dtype=int64)

In [29]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [30]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    labels = df["tag"].values
    
    labels = label_processor(labels[..., None]).numpy()

    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    for idx, path in enumerate(video_paths):
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels
train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

Frame features in train set: (18, 20, 2048)
Frame masks in train set: (18, 20)
train_labels in train set: (18, 1)
test_labels in train set: (9, 1)


In [31]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()


Epoch 1/30
1/1 [==============================] - ETA: 0s - loss: 1.1519 - accuracy: 0.3333
Epoch 1: val_loss improved from inf to 0.98336, saving model to ./tmp\video_classifier
1/1 [==============================] - 18s 18s/step - loss: 1.1519 - accuracy: 0.3333 - val_loss: 0.9834 - val_accuracy: 0.8333
Epoch 2/30
1/1 [==============================] - ETA: 0s - loss: 1.0128 - accuracy: 0.4167
Epoch 2: val_loss did not improve from 0.98336
1/1 [==============================] - 0s 84ms/step - loss: 1.0128 - accuracy: 0.4167 - val_loss: 1.0994 - val_accuracy: 0.0000e+00
Epoch 3/30
1/1 [==============================] - ETA: 0s - loss: 0.9891 - accuracy: 0.5833
Epoch 3: val_loss did not improve from 0.98336
1/1 [==============================] - 0s 84ms/step - loss: 0.9891 - accuracy: 0.5833 - val_loss: 1.1874 - val_accuracy: 0.0000e+00
Epoch 4/30
1/1 [==============================] - ETA: 0s - loss: 0.7782 - accuracy: 0.5833
Epoch 4: val_loss did not improve from 0.98336
1/1 [=======

In [32]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

  

Test video path: C:\Users\Hp\OneDrive\Desktop\human activity\data set\test/dancing/dance-3.mp4
  yoga: 37.33%
  dancing: 35.31%
  exercise: 27.36%
